# Task 2, Subtask 1

## Import libreries

In [1]:
import pandas as pd
import numpy as np
from sklearn.experimental import enable_hist_gradient_boosting  # noqa
from sklearn.ensemble import HistGradientBoostingClassifier
import sklearn.metrics as metrics
import time

## Start timer

In [2]:
start_time = time.time()

## Read data

In [3]:
## We read the already normalized and imputed data 
test_feat_path = "../data/test_features_imp.csv" 
train_feat_path = "../data/train_features_imp.csv" 
train_lab_path = "../data/train_labels.csv"
test_feat = pd.read_csv(train_feat_path)
train_feat = pd.read_csv(train_feat_path)
train_lab = pd.read_csv(train_lab_path)

## Order data to make sure that rows in X and Y match
test_feat.sort_values(by=['pid'], inplace = True, ignore_index = True)
train_feat.sort_values(by=['pid'], inplace = True, ignore_index = True)
train_lab.sort_values(by=['pid'], inplace = True, ignore_index = True)

## Get labels
labels_ids = ['LABEL_BaseExcess', 'LABEL_Fibrinogen', 'LABEL_AST',
              'LABEL_Alkalinephos', 'LABEL_Bilirubin_total', 
              'LABEL_Lactate', 'LABEL_TroponinI', 'LABEL_SaO2',
              'LABEL_Bilirubin_direct', 'LABEL_EtCO2']

## Select relevant label columns and exclude the pid column, and sample rows
X_test = train_feat.iloc[:, 1:272].values
X_train = train_feat.iloc[:, 1:272].values
Y_train = train_lab[labels_ids].to_numpy()

## Histogram-based Gradient Boosting Classification Tree

In [9]:
output = pd.DataFrame({'pid': test_feat.iloc[:, 0].values})
print("Validation scores for each label: ")
for i, label in enumerate(labels_ids):
    clf = HistGradientBoostingClassifier(scoring = 'f1_micro',
                                         validation_fraction = 0.2).fit(X_train, Y_train[:, i])
    print(clf.validation_score_[np.size(clf.validation_score_) - 1])
    output[label] = clf.predict_proba(X_test)[:, 1]

Validation scores for each label: 
0.8775993682548039
0.9376151618847065
0.799157673071861
0.7928402211108186
0.7925769939457752
0.8423269281389839
0.9307712555935772
0.8360094761779415
0.9686759673598315
0.9639378783890498


## Write probabilities to output

In [5]:
output

,pid,LABEL_BaseExcess,LABEL_Fibrinogen,LABEL_AST,LABEL_Alkalinephos,LABEL_Bilirubin_total,LABEL_Lactate,LABEL_TroponinI,LABEL_SaO2,LABEL_Bilirubin_direct,LABEL_EtCO2
0,1,0.818699,0.070224,0.079805,0.047943,0.102599,0.230049,0.006836,0.104406,0.010467,0.002933
1,2,0.034358,0.028550,0.143829,0.110638,0.188830,0.094281,0.589342,0.136889,0.021069,0.021622
2,4,0.062457,0.020703,0.244249,0.140089,0.179788,0.132640,0.170118,0.294691,0.020221,0.047154
3,6,0.914936,0.071789,0.098797,0.062465,0.083364,0.334616,0.007087,0.901355,0.013386,0.002443
4,8,0.013321,0.025056,0.172867,0.206602,0.183143,0.036787,0.255564,0.048866,0.019521,0.039455
...,...,...,...,...,...,...,...,...,...,...,...
18990,31653,0.552467,0.149032,0.306233,0.108901,0.107033,0.551873,0.010804,0.071723,0.012706,0.002523
18991,31654,0.016474,0.024168,0.165630,0.222148,0.165583,0.036741,0.053688,0.056842,0.020080,0.038602
18992,31656,0.910376,0.020472,0.100442,0.142707,0.116707,0.320739,0.008323,0.305013,0.023101,0.003310
18993,31657,0.045149,0.025991,0.147566,0.135522,0.132083,0.060434,0.039512,0.061834,0.016571,0.015503


In [6]:
#output.to_csv('../output/subtask_1_output', index = False, header = False)

## Runtime

In [7]:
print("--- %s minutes ---" % ((time.time() - start_time)/60))

--- 0.49489843448003135 minutes ---


## Results Log

|   | C | kernel | gamma | weight | features | n_features | F1 score | AUC | runtime (min) |
|---|---|---|---|---|---|---|---|---|---|
| run_1 |  1 |  rbf | scale  |  balanced |  median for NA's and mean  | 35 | 0.598165656150447 | ? | 33 |
| run_2 |  1 |  rbf | scale  |  balanced |  median for NA's and mean, max, min, median, sd  | 170 | 0.628216870267411 |?| 102 |
| run_3 |  1 |  rbf | scale  |  balanced |  median for NA's and mean, max, min, median, sd, range, skw, kurt  | 272 | 0.649372121402984 | 0.8236937992110356 | 141 |
| run_4 |  na |  na | na |  na |  median for NA's and mean, max, min, median, sd, range, skw, kurt  | 272 | 0.871097657278231 | 0.8222653647930391 | 0.5 |